In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import dot, stack, concatenate, exp, invlogit, logit

%load_ext lab_black

# Rats Example with Missing Data*

This example goes further into dealing with missing data in PyMC, including in the predictor variables.

Adapted from [Unit 8: ratsignorable1.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsignorable1.odc), [ratsignorable2.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsignorable2.odc), and [ratsinformative.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsinformative.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats.txt).

## Problem statement

We had a previous example about [dugongs](https://areding.github.io/6420-pymc/Unit6-dugongs.html) that dealt with missing data in the observed data (y values). This example shows how to deal with missing data in the input data (x). It's still pretty easy. You could look at it like creating another likelihood in the model, a very simple one where the observed data is x, and you use a single distribution to fill in the missing values (see ```x_imputed``` in the model below).

Original paper [here.](https://www.jstor.org/stable/pdf/2289594.pdf)

Gelfand et al 1990 consider the problem of missing data, and delete the last observation of cases 6-10, the last two from 11-20, the last 3 from 21-25 and the last 4 from 26-30.  The appropriate data file is obtained by simply replacing data values by NA (see below). The model specification is unchanged, since the distinction between observed and unobserved quantities is made in the data file and not the model specification.

In [2]:
x = np.array([8.0, 15.0, 22.0, 29.0, 36.0])

In [3]:
# import y data and create mask (missing data is represented as nan in the file)
y = np.loadtxt("../data/rats.txt")
y = np.nan_to_num(y, nan=-1)  # nan to -1
y = np.ma.masked_values(y, value=-1)  # create mask

## Model 1

This first model we only have missing data in our response variable (y). Notice that I made the shapes of alpha and beta (30, 1) instead of just 30. This is so that they broadcast correctly when combined (```mu = alpha + beta * x```). The NumPy docs have a helpful [page about broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html).


In [4]:
with pm.Model() as m:
    alpha_c = pm.Normal("alpha_c", 0, tau=1e-6)
    alpha_tau = pm.Gamma("alpha_tau", 0.001, 0.001)
    beta_c = pm.Normal("beta_c", 0, tau=1e-6)
    beta_tau = pm.Gamma("beta_tau", 0.001, 0.001)

    alpha = pm.Normal(
        "alpha", alpha_c, tau=alpha_tau, shape=(30, 1)
    )  # (30, 1) for broadcasting
    beta = pm.Normal("beta", beta_c, tau=beta_tau, shape=(30, 1))
    lik_tau = pm.Gamma("lik_tau", 0.001, 0.001)
    sigma = pm.Deterministic("sigma", 1 / lik_tau**0.5)

    mu = alpha + beta * x

    pm.Normal("likelihood", mu, tau=lik_tau, observed=y)

    trace = pm.sample(
        5000, tune=1000, init="jitter+adapt_diag_grad", target_accept=0.9
    )

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/model.py:1402: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_c, alpha_tau, beta_c, beta_tau, alpha, beta, lik_tau, likelihood_missing]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 91 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.


In [5]:
az.summary(
    trace,
    hdi_prob=0.95,
    var_names=["alpha_c", "alpha_tau", "beta_c", "beta_tau", "sigma"],
    kind="stats",
)

,mean,sd,hdi_2.5%,hdi_97.5%
alpha_c,101.245,2.301,96.599,105.669
alpha_tau,0.022,0.045,0.004,0.046
beta_c,6.565,0.165,6.239,6.885
beta_tau,2.863,1.245,0.924,5.227
sigma,6.031,0.662,4.845,7.394


## Model 2: Imputing missing predictor variable data

This is the same model, except we now have missing x data.

In [6]:
x_miss = np.array([8.0, 15.0, 22.0, -1, 36.0])
x_miss = np.ma.masked_values(x_miss, value=-1)

In [7]:
x_miss

masked_array(data=[8.0, 15.0, 22.0, --, 36.0],
             mask=[False, False, False,  True, False],
       fill_value=-1.0)

In [8]:
with pm.Model() as m:
    alpha_c = pm.Normal("alpha_c", 0, tau=1e-6)
    alpha_tau = pm.Gamma("alpha_tau", 0.001, 0.001)
    beta_c = pm.Normal("beta_c", 0, tau=1e-6)
    beta_tau = pm.Gamma("beta_tau", 0.001, 0.001)

    alpha = pm.Normal("alpha", alpha_c, tau=alpha_tau, shape=(30, 1))
    beta = pm.Normal("beta", beta_c, tau=beta_tau, shape=(30, 1))
    lik_tau = pm.Gamma("lik_tau", 0.001, 0.001)
    sigma = pm.Deterministic("sigma", 1 / lik_tau**0.5)

    x_imputed = pm.TruncatedNormal(
        "x_imputed", mu=20, sigma=10, lower=0, observed=x_miss
    )

    mu = alpha + beta * x_imputed

    pm.Normal("likelihood", mu, tau=lik_tau, observed=y)

    trace_2 = pm.sample(
        5000, tune=1000, cores=4, init="jitter+adapt_diag_grad", target_accept=0.9
    )

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/model.py:1402: ImputationWarning: Data in x_imputed contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/model.py:1402: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_c, alpha_tau, beta_c, beta_tau, alpha, beta, lik_tau, x_imputed_missing, likelihood_missing]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 699 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.


In [9]:
az.summary(
    trace_2,
    hdi_prob=0.95,
    var_names=["~alpha", "~beta", "~likelihood_missing"],
    kind="stats",
)

,mean,sd,hdi_2.5%,hdi_97.5%
alpha_c,101.836,2.335,97.067,106.267
beta_c,6.512,0.169,6.184,6.840
alpha_tau,1.716,24.920,0.003,0.082
beta_tau,2.945,1.259,1.003,5.419
lik_tau,0.029,0.006,0.017,0.042
...,...,...,...,...
"likelihood[29, 0]",153.000,0.000,153.000,153.000
"likelihood[29, 1]",198.624,9.680,179.844,217.769
"likelihood[29, 2]",244.052,12.344,219.714,267.852
"likelihood[29, 3]",292.678,16.013,260.663,323.681


## Model 3: Non-ignorable missingness

Probability of missingness increases approx. at a rate of 1% with increasing the weight.

In [10]:
y = np.atleast_2d(np.array([177.0, 236.0, 285.0, 350.0, -1]))  # original value was 320
y = np.ma.masked_values(y, value=-1)  # create masked array
# y.mask is equivalent to the "miss" array from the professor's example
miss = y.mask
x = np.array([8.0, 15.0, 22.0, 29.0, 36.0])

In [11]:
t = 0.1
s = 1 / t  # convert BUGS dlogis tau to s for pymc
b = np.log(1.01)

with pm.Model() as m:
    a = pm.Logistic("a", mu=0, s=s)
    alpha = pm.Flat("alpha")
    beta = pm.Flat("beta")
    log_sigma = pm.Flat("log_sigma")
    sigma2 = pm.Deterministic("sigma2", exp(2 * log_sigma))
    tau = pm.Deterministic("tau", 1 / sigma2)

    mu = pm.Deterministic("mu", alpha + beta * x)
    y_imputed = pm.Normal("likelihood", mu, tau=tau, observed=y)

    p = pm.Deterministic("p", invlogit(a + b * y_imputed))
    pm.Bernoulli("missing", p=p, observed=miss)

    trace_3 = pm.sample(4000, tune=2000, init="jitter+adapt_diag_grad")

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/model.py:1402: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, alpha, beta, log_sigma, likelihood_missing]


Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 20 seconds.
There were 148 divergences after tuning. Increase `target_accept` or reparameterize.


In [12]:
az.summary(trace_3, hdi_prob=0.95, kind="stats")

,mean,sd,hdi_2.5%,hdi_97.5%
a,-4.867,1.414,-7.820,-2.343
alpha,111.439,13.058,88.279,135.365
beta,8.150,0.671,6.917,9.293
log_sigma,1.807,0.610,0.755,3.033
likelihood_missing[0],405.777,19.316,377.068,435.457
sigma2,144.044,1334.789,2.729,356.261
tau,0.046,0.046,0.000,0.137
mu[0],176.635,8.458,161.501,192.384
mu[1],233.682,5.680,222.581,243.141
mu[2],290.728,6.096,280.759,300.845


In [13]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Wed Mar 22 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

pytensor: 2.10.1

pymc : 5.1.2
numpy: 1.24.2
arviz: 0.14.0

